![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Legal/2.Embeddings.ipynb)

# Legal Word and Sentence Embeddings

# Legal Word and Sentence Embeddings visualization using PCA (Principal Component Analysis)

Modern NLP models work with a numerical representation of texts and their menaning. For token classification problems (inferring a class for a token, for example Name Entity Recognition) Word Embeddings are required. For sentences, paragraph, document classification - we use Sentence Embeddings.

In this notebook, we use Spark NLP Legal Word (**roberta_embeddings_legal_roberta_base**) and Sentence (**sent_bert_base_uncased_legal**) Embeddings to get those numerical representations of the semantics of the texts. The result is a 768 embeddings matrix, impossible to process by the human eye.

There are many techniques we can use to visualize those embeddings. We are using one of them - Principal Component Analysis, a dimensionality reduction process, carried out by Spark MLLib. Both embeddings have 768 dimensions, so we will reduced this dimensions from **768** to **3** (X, Y, Z) and will use a color for the word / sentence legend.

## Import Required Library

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs 

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import * 
# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect
nlp.install()

## Start Spark Session

In [ ]:
from johnsnowlabs import * 
# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# if you want to start the session with custom params as in start function above
def start(SECRET):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+PUBLIC_VERSION) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+SECRET+"/spark-nlp-jsl-"+JSL_VERSION+".jar")
      
    return builder.getOrCreate()

#spark = start(SECRET)


# Get sample text

In [ ]:
! pip install plotly

# Downloading sample datasets.
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Legal/data/legal_pca_samples.csv



In [ ]:
df = pd.read_csv('legal_pca_samples.csv')

df.head()

,text,label
0,The fiscal year of the Company (herein called ...,fiscal-year
1,Each of the Borrower and each other member of ...,fiscal-year
2,Purchaser shall pay as the total Purchase Pric...,purchase-price
3,The purchase price to be paid by Purchaser to ...,purchase-price
4,The Guarantor hereby unconditionally and irrev...,guarantee


In [ ]:
# Create spark dataframe
sdf = spark.createDataFrame(df)
sdf.show()

+--------------------+--------------+
|                text|         label|
+--------------------+--------------+
|The fiscal year o...|   fiscal-year|
|Each of the Borro...|   fiscal-year|
|Purchaser shall p...|purchase-price|
|The purchase pric...|purchase-price|
|The Guarantor her...|     guarantee|
|The Holding Compa...|     guarantee|
|GFS will bear its...|      expenses|
|Each party shall ...|      expenses|
|Failure by either...|        waiver|
|Failure of any pa...|        waiver|
+--------------------+--------------+



# Sentence Embeddings

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = nlp.BertSentenceEmbeddings.pretrained("sent_bert_base_uncased_legal", "en") \
    .setInputCols("document") \
    .setOutputCol("document_embeddings")

sent_bert_base_uncased_legal download started this may take some time.
Approximate size to download 390.8 MB
[OK!]


# Custom transform to retrieve the numerical embeddings from Spark NLP and pass it to Spark MLLib

In [ ]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

In [ ]:
# This class extracts the embeddings from the Spark NLP Annotation object
# from pyspark import ml as ML

class EmbeddingsUDF(
    nlp.Transformer, nlp.ML.param.shared.HasInputCol,  nlp.ML.param.shared.HasOutputCol,
    nlp.ML.util.DefaultParamsReadable, nlp.ML.util.DefaultParamsWritable
):
    @keyword_only
    def __init__(self):
        super(EmbeddingsUDF, self).__init__()

        def _sum(r):
            result = 0.0
            for e in r:
                result += e
            return result

        self.udfs = {
            'convertToVectorUDF': F.udf(lambda vs: nlp.ML.linalg.Vectors.dense(vs), nlp.ML.linalg.VectorUDT()),
            'sumUDF': F.udf(lambda r: _sum(r), T.FloatType())
        }

    def _transform(self, dataset):

        results = dataset.select(
            "*", F.explode("document_embeddings.embeddings").alias("embeddings")
        )
        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [ ]:
embeddings_for_pca = EmbeddingsUDF()

In [ ]:
DIMENSIONS  = 3

In [ ]:
# import pyspark
pca = nlp.ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

### Full Spark NLP + Spark MLLib pipeline

In [ ]:
# We did all process in one pipeline

pipeline = nlp.Pipeline().setStages([document_assembler, embeddings, embeddings_for_pca, pca])

In [ ]:
pipeline.getStages()

[DocumentAssembler_ad39561f1987,
 BERT_SENTENCE_EMBEDDINGS_dae49a767331,
 EmbeddingsUDF_2adb779f64d7,
 PCA_fbe78dbf505e]

In [ ]:
model = pipeline.fit(sdf)

In [ ]:
result = model.transform(sdf)

In [ ]:
result.select('pca_features', 'label').show(truncate=False)

+------------------------------------------------------------+--------------+
|pca_features                                                |label         |
+------------------------------------------------------------+--------------+
|[-11.772444786459502,-3.1899426160082336,4.491155988811605] |fiscal-year   |
|[-11.401114580806883,-3.769737534360102,3.2405607816256428] |fiscal-year   |
|[-4.7833138649813876,-0.4942406903061646,2.8697672467860613]|purchase-price|
|[-5.455985032697512,-1.3412427647243466,3.3317233401952797] |purchase-price|
|[-8.841676655795245,-1.8203645840478873,0.13392410273877314]|guarantee     |
|[-11.532871191686569,-2.49957593411974,0.8141606865382868]  |guarantee     |
|[-5.731693818493869,-3.8158583926917053,3.5135882710427713] |expenses      |
|[-3.801045497461398,-4.3453619200612685,1.6774524605760983] |expenses      |
|[-6.783526297480713,-5.81545005442482,3.236173498846245]    |waiver        |
|[-7.1391811563595695,-6.340439897128405,1.2347292578808655] |wa

In [ ]:
df = result.select('pca_features', 'label').toPandas()

df
# As you see, dimension values are inside a list

,pca_features,label
0,"[-11.772444786459502, -3.1899426160082336, 4.4...",fiscal-year
1,"[-11.401114580806883, -3.769737534360102, 3.24...",fiscal-year
2,"[-4.7833138649813876, -0.4942406903061646, 2.8...",purchase-price
3,"[-5.455985032697512, -1.3412427647243466, 3.33...",purchase-price
4,"[-8.841676655795245, -1.8203645840478873, 0.13...",guarantee
5,"[-11.532871191686569, -2.49957593411974, 0.814...",guarantee
6,"[-5.731693818493869, -3.8158583926917053, 3.51...",expenses
7,"[-3.801045497461398, -4.3453619200612685, 1.67...",expenses
8,"[-6.783526297480713, -5.81545005442482, 3.2361...",waiver
9,"[-7.1391811563595695, -6.340439897128405, 1.23...",waiver


In [ ]:
# We extract the dimension values out off the list

df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["x", "y", "z", "label"]]

df

,x,y,z,label
0,-11.772445,-3.189943,4.491156,fiscal-year
1,-11.401115,-3.769738,3.240561,fiscal-year
2,-4.783314,-0.494241,2.869767,purchase-price
3,-5.455985,-1.341243,3.331723,purchase-price
4,-8.841677,-1.820365,0.133924,guarantee
5,-11.532871,-2.499576,0.814161,guarantee
6,-5.731694,-3.815858,3.513588,expenses
7,-3.801045,-4.345362,1.677452,expenses
8,-6.783526,-5.815450,3.236173,waiver
9,-7.139181,-6.340440,1.234729,waiver


In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x='x', y='y', z='z', color='label', width=800, height=600)

fig.show()

# Word Embeddings

We can also visualize the semantics of words, instead of full texts, by using Word Embeddings. We will add a Tokenizer and a WordEmbeddings model to get those embeddings, and them apply PCA as before.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols("document")\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"])\
    .setOutputCol("document_embeddings")

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[OK!]


In [ ]:
# Firstly we splitted the pipeline in two to get all token embeddings

pipeline = nlp.Pipeline().setStages([document_assembler, tokenizer, embeddings])

In [ ]:
model = pipeline.fit(sdf)

In [ ]:
result = model.transform(sdf)

In [ ]:
result_df = result.select("label", F.explode(F.arrays_zip("token.result", "document_embeddings.embeddings")).alias("cols"))\
                   .select(F.expr("cols['0']").alias("token"),
                           F.expr("cols['1']").alias("embeddings"),
                           "label")

result_df.show(truncate = 80)


+--------+--------------------------------------------------------------------------------+-----------+
|   token|                                                                      embeddings|      label|
+--------+--------------------------------------------------------------------------------+-----------+
|     The|[-0.19058353, 0.02907179, 0.13235606, 0.19562247, 0.77783114, 0.28990984, -0....|fiscal-year|
|  fiscal|[-0.19621773, 0.14509664, 0.23111394, -0.50601673, -0.38397044, -0.16950981, ...|fiscal-year|
|    year|[0.08006305, 0.22008368, 0.23202448, -0.4419725, 0.58936155, -0.23692255, 0.1...|fiscal-year|
|      of|[-0.14090274, 0.1561361, 0.24000195, -0.2449323, 0.897756, 0.4878102, 0.09172...|fiscal-year|
|     the|[-0.060954493, -0.08232107, 0.31499305, 0.12840052, -0.014585197, 0.97888094,...|fiscal-year|
| Company|[-0.06074696, 0.27488312, 0.07146063, -0.39569926, 0.73315394, 0.80515677, 0....|fiscal-year|
|       (|[0.110985324, -0.23188369, 0.11235473, 0.07458283, 0.8

In [ ]:
# Here we defined inheritance class from that defined previously EmbeddingsUDF class
class WordEmbeddingsUDF(EmbeddingsUDF):    
    def _transform(self, dataset):
        
        results = dataset.select('token', 'label', 'embeddings') # We changed this line because our embedding cloumn is already exploded

        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [ ]:
embeddings_for_pca = WordEmbeddingsUDF()

In [ ]:
DIMENSIONS  = 3

In [ ]:
# import pyspark
pca = nlp.ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

## Full Spark NLP + Spark MLLib pipeline

In [ ]:
# We run the second part of the pipeline

pipeline = nlp.Pipeline().setStages([embeddings_for_pca, pca])


In [ ]:
model = pipeline.fit(result_df)

In [ ]:
result = model.transform(result_df)

In [ ]:
result.select("token", "embeddings", "pca_features", "label").show(truncate = 60)

+--------+------------------------------------------------------------+------------------------------------------------------------+-----------+
|   token|                                                  embeddings|                                                pca_features|      label|
+--------+------------------------------------------------------------+------------------------------------------------------------+-----------+
|     The|[-0.19058353, 0.02907179, 0.13235606, 0.19562247, 0.77783...|   [6.574778635662345,7.652864315256421,-0.9220368677054456]|fiscal-year|
|  fiscal|[-0.19621773, 0.14509664, 0.23111394, -0.50601673, -0.383...| [5.227397542553504,-0.06940296725243511,-1.397650103795885]|fiscal-year|
|    year|[0.08006305, 0.22008368, 0.23202448, -0.4419725, 0.589361...|    [9.18560325370834,1.0014764546981136,1.0998418807415176]|fiscal-year|
|      of|[-0.14090274, 0.1561361, 0.24000195, -0.2449323, 0.897756...|   [6.397932452430552,3.191733260939934,-1.0405441035534406

In [ ]:
df = result.select('token', 'pca_features',  'label').toPandas()

df

,token,pca_features,label
0,The,"[6.574778635662345, 7.652864315256421, -0.9220...",fiscal-year
1,fiscal,"[5.227397542553504, -0.06940296725243511, -1.3...",fiscal-year
2,year,"[9.18560325370834, 1.0014764546981136, 1.09984...",fiscal-year
3,of,"[6.397932452430552, 3.191733260939934, -1.0405...",fiscal-year
4,the,"[7.77230332673885, 8.005475053468903, -2.27173...",fiscal-year
...,...,...,...
672,constitute,"[6.129946539768545, -1.5722424071914052, -0.58...",waiver
673,a,"[8.764682532303045, 1.0030824435174792, 0.4532...",waiver
674,continuing,"[4.150359748372968, -0.7722894817094628, -1.15...",waiver
675,waiver,"[7.8244290655055835, -1.474978942695142, -3.41...",waiver


In [ ]:
df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["token", "x", "y", "z", "label"]]

df

,token,x,y,z,label
0,The,6.574779,7.652864,-0.922037,fiscal-year
1,fiscal,5.227398,-0.069403,-1.397650,fiscal-year
2,year,9.185603,1.001476,1.099842,fiscal-year
3,of,6.397932,3.191733,-1.040544,fiscal-year
4,the,7.772303,8.005475,-2.271736,fiscal-year
...,...,...,...,...,...
672,constitute,6.129947,-1.572242,-0.582971,waiver
673,a,8.764683,1.003082,0.453227,waiver
674,continuing,4.150360,-0.772289,-1.152194,waiver
675,waiver,7.824429,-1.474979,-3.411841,waiver


In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = "label", width=1000, height = 800, hover_data = ["token", "label"])

fig.show()

That chart is super cool because you can see how the same token gets different embeddings depending on the context.